# Imports

In [1]:
from numpy.random import seed
seed(888)

from tensorflow import set_random_seed
set_random_seed(404)

C:\Users\Kanav\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Kanav\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Kanav\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Kanav\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [328]:
import os
import tensorflow as tf
import numpy as np
from time import strftime

from PIL import Image

# Constants

In [3]:
X_TRAIN_PATH='MNIST/digit_xtrain.csv'
X_TEST_PATH='MNIST/digit_xtest.csv'
Y_TRAIN_PATH='MNIST/digit_ytrain.csv'
Y_TEST_PATH='MNIST/digit_ytest.csv'
NUM_CLASSES=10
VALIDATION_SIZE=10000

LOGGING_PATH='tensor_mnist_digit_logs/'

IMAGE_HEIGHT=28
IMAGE_WIDTH=28
CHANNELS=1
TOTAL_INPUTS=IMAGE_HEIGHT*IMAGE_WIDTH*CHANNELS

# Fetch Data

In [4]:
y_train_all=np.loadtxt(Y_TRAIN_PATH, dtype='int', delimiter=',')
y_test=np.loadtxt(Y_TEST_PATH, dtype='int', delimiter=',')

In [5]:
%%time 

x_train_all=np.loadtxt(X_TRAIN_PATH, dtype=int, delimiter=',')

Wall time: 30.1 s


In [6]:
%%time
x_test=np.loadtxt(X_TEST_PATH, dtype=int, delimiter=',')

Wall time: 4.89 s


## Data Preprocessing

In [7]:
## The np.eye(n) is a diagonal matrix of diag elements = 1

x_test, x_train_all = x_test/255.0, x_train_all/255.0

y_test=np.eye(NUM_CLASSES)[y_test]
y_train_all=np.eye(NUM_CLASSES)[y_train_all]

In [8]:
y_train_all.shape

(60000, 10)

### Create Validation DataSet

In [9]:
x_val=x_train_all[:VALIDATION_SIZE]
x_train=x_train_all[VALIDATION_SIZE:]


y_val=y_train_all[:VALIDATION_SIZE]
y_train=y_train_all[VALIDATION_SIZE:]

## Setting up TensorFlow Graph

In [371]:
X=tf.placeholder(dtype=tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y=tf.placeholder(dtype=tf.float32, shape=[None, NUM_CLASSES],  name='labels')

### Neural Network Architecture

#### Hyperparameters

In [372]:
epochs = 50
learning_rate=0.001

n_hidden1=512
n_hidden2=64

In [373]:
def setup_layer(inp, weight_dim, bias_dim, name):
    with tf.name_scope(name):
        initial_w=tf.truncated_normal(shape=weight_dim, seed=42, stddev=0.1)
        w=tf.Variable(initial_w, name='W')
        
        initial_b=tf.constant(shape=bias_dim, value=0.0)
        b=tf.Variable(initial_b, name='W')

        layer_in=tf.matmul(inp, w) +b
        
        if name=='out':
            output=tf.nn.softmax(layer_in)
        else:
            output=tf.nn.relu(layer_in)
            
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        
    return output

In [374]:
# layer_1=setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], bias_dim=[n_hidden1], name='Layer_1')
# layer_2=setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2], bias_dim=[n_hidden2], name='Layer_2')
# output=setup_layer(layer_2, weight_dim=[n_hidden2, NUM_CLASSES], bias_dim=[NUM_CLASSES], name='out')

# model_name=f'{n_hidden1} - {n_hidden2} LR{learning_rate} E{epochs}' 

In [375]:
layer_1=setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], bias_dim=[n_hidden1], name='Layer_1')
layer_drop=tf.nn.dropout(layer_1, keep_prob=0.8)
layer_2=setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2], bias_dim=[n_hidden2], name='Layer_2')
output=setup_layer(layer_2, weight_dim=[n_hidden2, NUM_CLASSES], bias_dim=[NUM_CLASSES], name='out')

model_name=f'{n_hidden1} - {n_hidden2} LR{learning_rate} E{epochs}' 

## Tensorboard Setup

In [376]:
folder_name = f'{model_name} at {strftime("%H %M")}'
directory=os.path.join(LOGGING_PATH, folder_name)
try:
    os.makedirs(directory)
except OSError as e:
    print(str(e))
else:
    print('Successfully created directory')

Successfully created directory


## Loss, Optimisation, and Metrics

In [377]:
## Cross entropy loss function shall be used
with tf.name_scope('loss_calc'):
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output , labels=Y)) ##defining loss fn
## defining the optimiser we gonna use
with tf.name_scope('optimizer'):
    optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_step=optimizer.minimize(loss)## defining the parameter that optimizer minimises


### Accuracy Metric

In [378]:
with tf.name_scope('accuracy_calc'):
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy= tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [379]:
with tf.name_scope('performance'):

    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

### Check Input Images in Tensorboard

In [380]:
with tf.name_scope('Show_Image'):

    x_image=tf.reshape(X, [-1,28,28,1])

    tf.summary.image('image_input', x_image, max_outputs=4)

##  Running Session

In [381]:
sess=tf.Session()

init=tf.global_variables_initializer()
sess.run(init)

### Setup File Writer and Merging Summaries

In [382]:
merged_summary=tf.summary.merge_all()

train_writer=tf.summary.FileWriter(directory+'/train')
train_writer.add_graph(sess.graph)

validation_writer=tf.summary.FileWriter(directory+'/validation')

In [383]:
size_of_batch=1000

num_examples=y_train.shape[0]
num_iterations = int(num_examples/size_of_batch)

index_in_epoch=0

In [384]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start=index_in_epoch
    index_in_epoch+=batch_size
    
    if index_in_epoch>num_examples:
        start=0
        index_in_epoch=batch_size
    
    end=index_in_epoch
    
    return data[start: end], labels[start:end]

### Training Model

In [385]:
for epoch in range(epochs):
    #------Training DAtaset------------#
    for i in range(num_iterations):
        
        batch_x, batch_y=next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        
        feed_dict= {X: batch_x, Y: batch_y}
        sess.run(train_step, feed_dict=feed_dict)
        
    s, batch_accuracy=sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dict)
    
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch}   | Training Accuracy: {batch_accuracy}')
    
    #------Validation Dataset-----------#
    
    summary=sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch) 
print('Training Done')

Epoch 0   | Training Accuracy: 0.8500000238418579
Epoch 1   | Training Accuracy: 0.8600000143051147
Epoch 2   | Training Accuracy: 0.8679999709129333
Epoch 3   | Training Accuracy: 0.8700000047683716
Epoch 4   | Training Accuracy: 0.8730000257492065
Epoch 5   | Training Accuracy: 0.9700000286102295
Epoch 6   | Training Accuracy: 0.9739999771118164
Epoch 7   | Training Accuracy: 0.9819999933242798
Epoch 8   | Training Accuracy: 0.9829999804496765
Epoch 9   | Training Accuracy: 0.9819999933242798
Epoch 10   | Training Accuracy: 0.9829999804496765
Epoch 11   | Training Accuracy: 0.9879999756813049
Epoch 12   | Training Accuracy: 0.9869999885559082
Epoch 13   | Training Accuracy: 0.9879999756813049
Epoch 14   | Training Accuracy: 0.9900000095367432
Epoch 15   | Training Accuracy: 0.9890000224113464
Epoch 16   | Training Accuracy: 0.9909999966621399
Epoch 17   | Training Accuracy: 0.9900000095367432
Epoch 18   | Training Accuracy: 0.9900000095367432
Epoch 19   | Training Accuracy: 0.9909999

## Making a prediction

In [386]:
img=Image.open('MNIST/test_img.png')
bw= img.convert('L')
img_array=np.invert(bw)
img_array.shape
test_img=img_array.ravel()


In [387]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X: [test_img]})

In [388]:
type(prediction[0])

numpy.int64

In [389]:
img_array.shape

(28, 28)

## Testing and Evaluation

In [390]:
x_test.shape

(10000, 784)

In [391]:
y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [392]:
np.argmax(y_test[1])==prediction[0]

True

In [393]:
test_accuracy=sess.run(fetches=accuracy, feed_dict={X: x_test, Y: y_test})
print(f'Accuracy on test set is {test_accuracy: 0.2%}')

Accuracy on test set is  97.78%


## Resetting for the next run

In [394]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()

In [395]:
## Code for first part of the module

In [396]:
# with tf.name_scope('hidden_layer_1'):
#     initial_w1=tf.truncated_normal(shape=(TOTAL_INPUTS, n_hidden1), seed=42, stddev=0.1)
#     w1=tf.Variable(initial_w1, name='w1')

#     initial_b1=tf.constant(shape=[n_hidden1], value=0.0)
#     b1=tf.Variable(initial_b1, name='b1')

#     layer1_in = tf.matmul(X, w1) + b1
#     layer1_out= tf.nn.relu(layer1_in)

In [397]:
# with tf.name_scope('hidden_layer_2'):
#     initial_b2=tf.constant(shape=[n_hidden2], value=0.0)
#     b2=tf.Variable(initial_b2, name='b2')

#     initial_w2=tf.truncated_normal(shape=(n_hidden1, n_hidden2), seed=42, stddev=0.1)
#     w2=tf.Variable(initial_w2, name='w2')

#     layer2_in=tf.matmul(layer1_out, w2) + b2
#     layer2_out=tf.nn.relu(layer2_in)

# with tf.name_scope('output_layer'):
#     initial_w_out=tf.truncated_normal(shape=(n_hidden2, NUM_CLASSES), seed=42, stddev=0.1)
#     w_out=tf.Variable(initial_w_out, name='w_out')

#     initial_b_out=tf.constant(shape=[NUM_CLASSES], value=0.0)
#     b_out=tf.Variable(initial_b_out, name='b_out')

#     layer3_in=tf.matmul(layer2_out, w_out) +b_out
#     output=tf.nn.softmax(layer3_in)